# Videogame Sales Case

## Setup

### Libraries || Bibliotecas

### Recurring Functions || Funções de uso Recorrente

## Data Description

- **Name**: The name of the video game.
- **Platform**: The platform the game was released on.
- **Year_of_Release**: The release year of the game.
- **Genre**: The genre of the game.
- **NA_sales**: North American sales in millions of USD.
- **EU_sales**: European sales in millions of USD.
- **JP_sales**: Japanese sales in millions of USD.
- **Other_sales**: Sales in other regions in millions of USD.
- **Critic_Score**: Critic's score (maximum of 100).
- **User_Score**: User's score (maximum of 10).
- **Rating**: ESRB rating (Entertainment Software Rating Board).

---
---

## Descrição dos Dados

- **Name**: Nome do jogo.
- **Platform**: Plataforma em que o jogo foi lançado.
- **Year_of_Release**: Ano de lançamento do jogo.
- **Genre**: Gênero do jogo.
- **NA_sales**: Vendas na América do Norte em milhões de USD.
- **EU_sales**: Vendas na Europa em milhões de USD.
- **JP_sales**: Vendas no Japão em milhões de USD.
- **Other_sales**: Vendas em outras regiões em milhões de USD.
- **Critic_Score**: Pontuação dos críticos (máximo de 100).
- **User_Score**: Pontuação dos usuários (máximo de 10).
- **Rating**: Classificação ESRB (Entertainment Software Rating Board).

## Step 1 || Etapa 1
- Opening the file, taking first conclusions and tracing a roadmap for the study
---
- Abrindo o arquivo de dados, tirando primeiras conclusões e traçando a rota para o estudo do caso

### 1.1 Conclusions || Conclusões

## Step 2 || Etapa 2

### 2.1 Cleaning and formatting || Limpando e formatando

### 2.2 Adding computed Data || Adicionando dados computados

### 2.3 Conclusions || Conclusões

## Step 3 || Etapa 3

### Conclusions || Conclusões

## Step 4 || Etapa 4

### Conclusions || Conclusões

## Step 5 || Etapa 5

### Conclusions || Conclusões

## Final Conclusion || Conclusão Final